In [1]:
from datetime import datetime, timedelta
import sys
import os
import pandas as pd
import numpy as np
colony = 'worker05'
date_tagging = [ [2024, 3, 14], [2024, 4, 11] ] 
crop_datetime = date_tagging[0]

In [2]:
def parameters_list_crop_by_datetime(lines, crop_date):
    # initialize lists
    ID_num = []
    center_x = []
    center_y = []
    angle = []
    date_time = []
    CPU_temp = []

    for line in lines:
        # replace unnecessary characters in line
        line = line.replace('[','')
        line = line.replace(']','')
        line = line.replace('CPU','')
        
        # replace month name with month number so that it can be converted to datetime type
        months = ['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec']
        for month in range(0,len(months)):
            line = line.replace(months[month],str(month+1))
            
        # split line by spaces
        line = line.split()
        
        # extract info and convert to usable types
        ID_num.append(int(line[6]))
        center_x.append(float(line[8]))
        center_y.append(float(line[9]))
        angle.append(float(line[11]))
        CPU_temp.append(float(line[13]))
        
        # record dates and times as datetime type for calculations
        (h, m, s) = line[3].split(':')
        line_time = datetime(int(line[4]),int(line[1]),int(line[2]),int(h),int(m),int(s))
        date_time.append(line_time)
    
    
    for index in range(0,len(date_time)):
        if date_time[index] >= datetime(crop_date[0], crop_date[1], crop_date[2]):
            crop_date_index = index
            break
    
    
    
    return ID_num[crop_date_index::], center_x[crop_date_index::], center_y[crop_date_index::], angle[crop_date_index::], date_time[crop_date_index::], CPU_temp[crop_date_index::]
    

In [3]:
with open(colony + '.txt') as file: # $ change file name if necessary 
    lines = file.readlines()
    file.close()

ID_num, center_x, center_y, angle, date_time, CPU_temp = parameters_list_crop_by_datetime(lines, crop_datetime)


In [4]:
vdf = pd.DataFrame.from_dict({'track_tagid':ID_num,'x':center_x,'y':center_y,'datetime':date_time,'angle':angle})
vdf

,track_tagid,x,y,datetime,angle
0,8568,31.772220,260.044083,2024-03-14 19:32:53,270.888823
1,3972,238.394828,246.301779,2024-03-15 00:44:58,95.547329
2,2988,20.697819,232.735955,2024-03-15 04:14:38,252.737376
3,119,733.835494,23.445411,2024-03-15 09:43:19,308.889661
4,119,744.773536,24.291000,2024-03-15 09:43:23,323.908362
...,...,...,...,...,...
17662,5183,302.810705,194.953068,2024-05-05 16:37:06,270.897081
17663,5183,340.562724,271.436619,2024-05-05 16:37:21,99.913396
17664,5017,693.357921,63.091742,2024-05-05 16:42:14,94.338477
17665,5017,703.129587,46.054886,2024-05-05 16:42:14,83.400180


In [5]:
max(vdf['y'])

316.2050042

In [6]:
#SECONDS THRESHOLD
#used to classify multiple detections into part of a single event
#also used to tell when two detections are part of different events
#by checking the time distance between them
t = 15

#DISTANCE THRESHOLD
#used to classify an event as entering or exiting
#when two consecutive detections of the same event 
#have this distance in y position, they are utilized to predict
#the trajectory. If not then it checks the detection prior for
#the distance threshold, and continues doing so until it finds
#the last detection in the event or until it finds a distance
#of more than the threshold

t2 = 10

#ANGLE THRESHOLD
#used to generate angle ranges for classifying as exiting
#or entering

angle = 30


## Modified look-at prior events for this type of data

In [7]:
def beeCleanPrior(bee):

    id = bee['track_tagid'].iloc[0]

    new_event = []
    datetime = []
        
    for i in range(len(bee)-1):

            time = bee['datetime'].iloc[i]
            next_t = bee['datetime'].iloc[i+1]

            init = bee['y'].iloc[i]

            #if time passed is greater than the threshold,
            #classify the last detection in an event
            if (next_t - time).total_seconds() > t:
                second = bee['y'].iloc[i]
                matched = False
                counter = 1
                #iterate backwards until the distance threshold
                #is met or until they reach the first detection in
                #the event
                while not matched: 
                    if abs(second-init) > t2:
                        if second > init:
                            new_event.append('exiting')
                        else:
                            new_event.append('entering')
                        matched = True
                    elif counter < i and (time - bee['datetime'].iloc[i-counter]).total_seconds() < t:
                        init = bee['y'].iloc[i-counter]
                        counter += 1
                    else:
                        if second > init:
                            new_event.append('exiting')
                        elif init > second:
                            new_event.append('entering')
                        else:
                            new_event.append('unknown')
                        matched = True
                datetime.append(time)
                
                
            
    tagID = [id] * len(new_event)
    df = pd.DataFrame.from_dict({'tagID': tagID, 'datetime':datetime, 'event':new_event})
    return df

In [8]:
bees = []

beeIDs = vdf['track_tagid'].unique()
for bee in beeIDs:

    b = vdf[vdf['track_tagid'] == bee].copy().reset_index()
    events = beeCleanPrior(b)
    bees.append(events)
    
    
new = pd.concat(bees, axis = 0) 
new['tagID'] = new['tagID'].apply(lambda x: int(x))
new.to_csv("beecam-prior.csv", index=False)
prior = new

C:\Users\kashi\AppData\Local\Temp\ipykernel_8336\2779204676.py:11: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  new = pd.concat(bees, axis = 0)


In [9]:
len(new[new['event'] == 'unknown'])/len(new)

0.36401212455221826

## Angle based from BeeCam-AprilTag

In [10]:
def beeCleanAngle(bee):

    id = bee['track_tagid'].iloc[0]
    new_event = []
    datetime = []
    vector_x = []
    vector_y = []

    
    exit_min = 180 + angle
    exit_max = 360 - angle
    enter_min = angle
    enter_max = 180 - angle
        
    
    for i in range(len(bee)-1):

            time = bee['datetime'].iloc[i]
            next_t = bee['datetime'].iloc[i+1]
            angles = bee['angle'].iloc[i+1]

            #obtain direction vector 
            #normalized to a magnitude of 1
            dx = np.cos(np.deg2rad(angles)) 
            dy = np.sin(np.deg2rad(angles))
            angle_deg = np.rad2deg(np.arctan2(dy, dx))
            unit_dx = np.cos(np.deg2rad(angle_deg))
            unit_dy = np.sin(np.deg2rad(angle_deg))

            vector_x.append(unit_dx)
            vector_y.append(unit_dy)
            
            #if next detection is further than the threshold,
            #utilize all stored angles to calculate event trajectory
            if (next_t - time).total_seconds() > t:
                #obtain average
                avg_x = sum(vector_x)/len(vector_x)
                avg_y = sum(vector_y)/len(vector_y)

                if avg_x == 0 and avg_y == 0:
                    deg = 0
                elif avg_x == 0 and avg_y != 0:
                    if avg_y > 0:
                        deg = 270
                    elif avg_y < 0:
                        deg = 90
                else:
                    # determine direction angle using arctan
                    deg = np.rad2deg(np.arctan(avg_y/avg_x))
                    
                    # since arctan limits are (-90,90), use coordinate directions to 
                    # correct the angle to be within standard [0,360) range
                    if avg_x > 0 and avg_y >= 0:
                        deg = deg
                    elif avg_x < 0 and avg_y >= 0:
                        deg = 180 + deg
                    elif avg_x < 0 and avg_y < 0:
                        deg = deg + 180
                    elif avg_x > 0 and avg_y < 0:
                        deg = 360 + deg

                #classify events based on angle threshold
                if deg >= exit_min and deg <= exit_max:
                    new_event.append('exiting')
                elif deg >= enter_min and deg <= enter_max:
                    new_event.append('entering')
                else:
                    new_event.append('unknown')
                datetime.append(time)
                vector_x = []
                vector_y = []
            
    tagID = [id] * len(new_event)
    df = pd.DataFrame.from_dict({'tagID': tagID, 'datetime':datetime, 'event':new_event})
    #print(bee)
    return df

In [11]:
bees = []

beeIDs = vdf['track_tagid'].unique()
for bee in beeIDs:

    b = vdf[vdf['track_tagid'] == bee].copy().reset_index()
    events = beeCleanAngle(b)
    #print(events)
    bees.append(events)
    
    
new = pd.concat(bees, axis = 0) 
new['tagID'] = new['tagID'].apply(lambda x: int(x))
new.to_csv("beecam-angle.csv", index=False)
angles = new

C:\Users\kashi\AppData\Local\Temp\ipykernel_8336\2778133797.py:12: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  new = pd.concat(bees, axis = 0)


In [12]:
len(new[new['event'] == 'unknown'])/len(new)

0.17139707908514742

In [13]:
len(prior[(prior['event'] == angles['event']) & (prior['event'] != 'unknown') & (angles['event'] != 'unknown')])/len(prior[(prior['event'] != 'unknown') & (angles['event'] != 'unknown')])

0.8591625883632409